# Llama Fine-tuning Libraries

#### TorchTune for Llama fine-tuning
- Based on configurable templates
- Ideal for: scaling quickly
<br/><br/>

#### SFTTrainer from Hugging Face
- Access to other LLMs
- Ideal for: fine-tuning multiple models
<br/><br/>

#### Unsloth
- Efficient memory usage
- Ideal for: limited hardware
<br/><br/>

#### Axolotl
- Modular approach
- Ideal for: no extensive reconfiguration

---

<br/>

### TorchTune Receipes

In [4]:
!tune ls

import error: No module named 'triton'
W0318 09:24:16.595000 44671 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         qwen2/7B_full_si

In [ ]:
# Example of running file-tuning
# !tune run full_finetune_single_device --config \llama3_1/8B_lora_single_device

---

<br/>

### Loading dataset from Hugging face

In [10]:
from datasets import load_dataset, Dataset

ds = load_dataset(
  "bitext/Bitext-customer-support-llm-chatbot-training-dataset",
  split="train"
)

print(ds.column_names)

['flags', 'instruction', 'category', 'intent', 'response']


Peeking into the data

In [11]:
import pprint

pprint.pprint(ds[0])

{'category': 'ORDER',
 'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order "
             "{{Order Number}}, and I'm here to provide you with the "
             'information you need. Please go ahead and ask your question, and '
             "I'll do my best to assist you."}


Filtering the dataset

In [15]:
first_thousand_points = ds[:1000]
filtered_ds = Dataset.from_dict(first_thousand_points)

filtered_ds

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 1000
})

Preprocessing the dataset

In [19]:
def merge_example(row):
  row['conversation'] = f"Query : {row['instruction']}\nResponse : {row['response']}"
  return row

filtered_ds = filtered_ds.map(merge_example)
print(filtered_ds[0]["conversation"])

Map: 100%|██████████| 1000/1000 [00:00<00:00, 27914.01 examples/s]

Query : question about cancelling order {{Order Number}}
Response : I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


Saving the preprocessed dataset

In [20]:
filtered_ds.save_to_disk("preprocessed_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 194513.94 examples/s]


Loading preprocessed dataset

In [21]:
from datasets import load_from_disk

ds_preprocessed = load_from_disk("preprocessed_dataset")
print(ds_preprocessed[0]["conversation"])

Query : question about cancelling order {{Order Number}}
Response : I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


---

<br/>

### Fine tuning with torchtune

Custom Receipe

In [22]:
import yaml

config_dict = {
    "model":{
        "_component_" : "torchtune.models.llama3_2.llama3_2_1b"
    },
    "batch_size":8,
    "device":"cuda",
    "epochs": 15,
    "optimizer": {"_component_": "bitsandbytes.optim.PagedAdamW8bit", "lr": 3e-05},
    "dataset": {"_component_": "custom_dataset"},
    "output_dir": "/tmp/finetune_results"
}

yaml_file_path = "custom_receipe.yaml"
with open(yaml_file_path, "w") as file:
  yaml.dump(config_dict, file)

In [ ]:
!tune run --config custom_receipe.yaml